In [1]:
from pprint import pprint
import os
import weaviate
from transformers import AutoTokenizer, AutoModel
import torch
from core.local_llms import llm_request, prompt_generator, print_out_ans

/Users/anaconda3/envs/tungtk/lib/python3.10/site-packages/llama_index/download/module.py:12: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


### 02. TRUY VẤN VÀ TƯƠNG TÁC VỚI CSDL

In [5]:
def query_weaviate(user_query, class_name, property_name="content_txt", limit=5):
    try:
        # Set up connection to Weaviate
        client = weaviate.Client("http://203.113.132.109:8080")

        # Use Weaviate to search with a limit of specified results
        query_result = client.query.get(
            class_name,
            [property_name]
        ).with_near_text({
            "concepts": [user_query]
        }).with_limit(limit).do()

        # Check if 'data' key is in response
        if 'data' not in query_result or class_name not in query_result['data']['Get']:
            print("No 'data' key in response or class_name not found in response.")
            return None

        # Store the results in a variable
        hints = [item[property_name] for item in query_result["data"]["Get"][class_name]]

        # Return the results
        return hints

    except weaviate.UnexpectedStatusCodeException as e:
        print("Error during Weaviate search:", e)
        return None
    except KeyError as e:
        print(f"KeyError occurred: {e}")
        return None


#---------------------------------------------------------
# Define Params
weaviate_url = "http://203.113.132.109:8080"
client = weaviate.Client(url=weaviate_url)

# Define the user query and class name
user_query = "paic là công ty nào? Trả lời bằng tiếng Việt"
class_name = "Document"

#---------------------------------------------------------
# Run the query function and store the hints
hints = query_weaviate(user_query, class_name)

#---------------------------------------------------------
# Create prompt
prompt = prompt_generator(hints, user_query)
#---------------------------------------------------------
# Create answer
final_ans = llm_request(prompt)
#---------------------------------------------------------
# Print Answer
answers_print = print_out_ans(user_query, final_ans,hints)
print(answers_print)



    Question: paic là công ty nào? Trả lời bằng tiếng Việt
    Answer:  PAIC là Công ty Cổ phần Công nghệ Thông tin, Viễn thông và Tự động hóa Dầu khí (Petroleum Automation Technology, Information and Communication Corporation).
    


In [ ]:
prompt = """
You are an AI assistant designed by the Vietnam Petroleum Institute. Answer the given question based on the context below. If the context doesn't have enough information, 
please reply "I don't have enough information"
    Context:

Hworld was open for Việt Minh forces to receive additional supplies which would allow them to escalate the fight against the French regime throughout Indochina. At the outset of the conflict, Hồ reportedly told a French visitor: "You can kill ten of my men for every one I kill of yours. But even at those odds, you will lose and I will win".[77] In 1954, the First Indochina War came to an end after the decisive Battle of Dien Bien Phu, where more than 10,000 French soldiers surrendered to the Viet Minh. The subsequent
by Chiang Kai-shek's local authorities before being rescued by Chinese Communists.[49] Following his release in 1943, he returned to Vietnam. It was during this time that he began regularly using the name Hồ Chí Minh, a Vietnamese name combining a common Vietnamese surname (Hồ, 胡) with a given name meaning "Bright spirit" or "Clear will" (from Sino-Vietnamese 志 明: Chí meaning "will" or "spirit" and Minh meaning "bright").[10]: 248–249 His new name was a tribute to General Hou Zhiming (侯志明), Chief Commissar of
began arriving in South Vietnam, first to protect the airbases around Chu Lai and Da Nang, later to take on most of the fight as "[m]ore and more American troops were put in to replace Saigon troops who could not or would not, get involved in the fighting".[120] As fighting escalated, widespread aerial and artillery bombardment all over North Vietnam by the United States Air Force and Navy began with Operation Rolling Thunder. On 8–9 April 1965, Hồ made a secret visit to Beijing to meet Mao Zedong.[121] It was
praises the legacy of Uncle Hồ (Bác Hồ), the Bringer of Light (Chí Minh). Although Hồ Chí Minh wished for his body to be cremated and his ashes spread to North, Central, and South Vietnam, the body instead is embalmed on view in a mausoleum. His image is featured in many public buildings and schoolrooms, and other displays of reverence.[149] There is at least one temple dedicated to him, built in then Việt Cộng-controlled Vĩnh Long shortly after his death in 1970.[150] Hồ Chí Minh statue and a yellow star as
outlook emerged,[citation needed] but his application to the French Colonial Administration School in 1911 undermines this version of events, in which he stated that he left school to go abroad. Because his father had been dismissed, he no longer had any hope for a governmental scholarship and went southward, taking a position at Dục Thanh school in Phan Thiết for about six months, then traveled to Saigon.[citation needed] Overseas sojourn In France Commemorative plaque in Haymarket in London In Saigon, he applied
which now stands on the site of the Carlton Hotel, displays a blue plaque. During 1913, Thành was also employed as a pastry chef on the Newhaven–Dieppe ferry route.[25] Political education in France Nguyễn Ái Quấc's identity card issued by the French government in 1919 Hồ Chí Minh, 1921, going by the pseudonym Nguyễn Ái Quốc, attending a Communist congress in Marseille, France. From 1919 to 1923, Thành (Hồ) began to show an interest in politics while living in France, being influenced by his friend and Socialist
and the Việt Cộng during the Vietnam War, which lasted from 1955 to 1975. Hồ officially stepped down from power in 1965 due to health problems and died in 1969. North Vietnam was ultimately victorious against South Vietnam and its allies. Vietnam was officially unified in 1976. Saigon, the former capital of South Vietnam, was renamed Ho Chi Minh City in his honor. The details of Hồ Chí Minh's life before he came to power in Vietnam are uncertain. He is known to have used between 50[10]: 582 and 200 pseudonyms.[11]
on, Hồ Chí Minh and his government's strategy, based on the idea of not using conventional warfare and facing the might of the United States Army, which would wear them down eventually while merely prolonging the conflict, would lead to the eventual acceptance of Hanoi's terms, materialized. In early 1969, Hồ suffered a heart attack and was in increasingly bad health for the rest of the year.[134] In July 1969, Jean Sainteny, a former French official in Vietnam who knew Hồ secretly relayed a letter written to
least among these, of "total mobilization in the struggle for national independence and state power."[157] This "congeniality" with Confucian tradition was remarked on by Nguyen Khac Vien, a leading Hanoi intellectual of the 1960s and 70s. In Confucianism and Marxism in Vietnam[158] Nguyen Khac Vien, saw definite parallels between Confucian and party discipline, between the traditional scholar gentry and Hồ Chí Minh's party cadres.[159] A completely different form of the cult of Hồ Chí Minh (and one tolerated
Asia, Europe, America and Africa have erected monuments or statues in remembrance of Hồ Chí Minh.[168] International influence Hồ Chí Minh bust in Kolkata, India Hồ Chí Minh is considered one of the most influential leaders in the world. Time magazine listed him in the list of 100 Most Important People of the Twentieth Century (Time 100) in 1998.[169][170] His thought and revolution inspired many leaders and people on a global scale in Asia, Africa and Latin America during the decolonization movement which occurred
1957 visit to East Germany Hồ Chí Minh with members of the East German Young Pioneers near Berlin, 1957 Between 1953 and 1956, the North Vietnamese government instituted various agrarian reforms, including "rent reduction" and "land reform", which were accompanied by political repression. During the land reform, testimonies by North Vietnamese witnesses suggested a ratio of one execution per 160 village residents, which if extrapolated would indicate a nationwide total of nearly 100,000 executions. Because the
that he wanted to be cremated.[10]: 565 The North Vietnamese government originally announced Hồ's death on 3 September. A week of mourning for his death was decreed nationwide in North Vietnam from 4 to 11 September 1969.[147] His funeral was attended by about 250,000 people and 5,000 official guests, which included many international mourners. Representatives from 40 countries and regions were also presented. During the mourning period, North Vietnam received more than 22,000 condolences letters from 20 organizations
...
Liên commune, Nam Đàn district, Nghệ An province, in Central Vietnam which was then a French protectorate. Although 1890 is generally accepted as his birth year, at various times he used four other birth years:[13][page needed] 1891,[14] 1892,[k] 1894[l] and 1895.[15] He lived in his father Nguyễn Sinh Sắc's village of Làng Sen in Kim Liên until 1895 when his father sent him to Huế for study. He had three siblings: his sister Bạch Liên (Nguyễn Thị Thanh), a clerk in the French Army; his brother Nguyễn Sinh Khiêm




    Question:
    what did happend in 1890?
    Assistant:
"""
# Create answer
final_ans = llm_request(prompt)
#---------------------------------------------------------
# Print Answer
answers_print = print_out_ans(user_query, final_ans,hints)
print(answers_print)